# 📚 Импорт библиотек

In [2]:
import requests
import time
from datetime import datetime
import pandas as pd
from tqdm import tqdm

import pickle

# 👛 Импорт адресов

In [3]:
df = pd.read_pickle("df_addresses.pickle")
df

,index,user,airdropped_tokens,get_drop
0,0,0x934204e4617792b8ec2a34f0330a19ab3393041f,5.464521,0
1,1,0xa99979dca47c7c41262936521d59d9a922f7a1d2,4.161325,0
2,2,0x5fc7b11e3281b7404573465bc99d9aebc87ec59b,5.756350,0
3,3,0x7c56d095261c0c9921adf95212f499018340bdde,19.357885,1
4,4,0xa33f601b04ba846586266bfae01ae47d5362eff4,6.113811,1
...,...,...,...,...
745228,745228,0x5e73cf9cebd82d8c192f538f41186b55965e6fc6,25.185039,1
745229,745229,0x335b7c6633a29e43dbe2f7804803cc8243f2a830,65.942124,1
745230,745230,0xecebbda750e02674eab8929c69e9de5b30c4446f,26.084444,1
745231,745231,0x45d1d1124a10dafdf0fe6c6e12a1dea9c1c6db25,159.581385,1


In [4]:
df_15k = df[df["index"] < 740000]
list_of_addresses_15k = list(df_15k["user"])

# 🧰 Нужные функции

## Перевод строки в UNIX-timestamp 

In [5]:
def date_to_timestamp(date_str: str) -> int:
    """
    Переводит дату вида 'YYYY-MM-DD' в UNIX‑таймстамп (секунды).
    """
    dt = datetime.strptime(date_str, "%Y-%m-%d")
    return int(time.mktime(dt.timetuple()))

## Функции сбора транзакций и тарнсферов

In [6]:
def fetch_transactions(
    address: str, api_key: str, end_date: str, tx_type: str = "normal"
) -> list:
    """
    Собирает все транзакции для address до end_date с Etherscan API.
    tx_type: 'normal' или 'internal'.
    """
    base_url = "https://api.etherscan.io/api"
    action_map = {"normal": "txlist", "internal": "txlistinternal"}
    action = action_map.get(tx_type, "txlist")
    cutoff_ts = date_to_timestamp(end_date)

    params = {
        "module": "account",
        "action": action,
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "page": 1,
        "offset": 10000,
        "sort": "asc",
        "apikey": api_key,
    }

    result_txs = []
    while True:
        resp = requests.get(base_url, params=params)
        data = resp.json()
        # Проверяем успешность ответа и наличие списка транзакций
        if data.get("status") != "1" or not isinstance(data.get("result"), list):
            break

        txs = data["result"]
        for tx in txs:
            ts = int(tx.get("timeStamp", 0))
            if ts <= cutoff_ts:
                result_txs.append(tx)
            else:
                return result_txs

        # Если пришло меньше, чем запросили, значит это последняя страница
        if len(txs) < params["offset"]:
            break
        params["page"] += 1

    return result_txs


def fetch_token_transfers(address: str, api_key: str, end_date: str) -> list:
    """
    Собирает все ERC-20 переводы для address до end_date с Etherscan API.
    """
    base_url = "https://api.etherscan.io/api"
    cutoff_ts = date_to_timestamp(end_date)

    params = {
        "module": "account",
        "action": "tokentx",
        "address": address,
        "startblock": 0,
        "endblock": 99999999,
        "page": 1,
        "offset": 10000,
        "sort": "asc",
        "apikey": api_key,
    }

    transfers = []
    while True:
        resp = requests.get(base_url, params=params)
        data = resp.json()
        if data.get("status") != "1" or not isinstance(data.get("result"), list):
            break

        batch = data["result"]
        for tx in batch:
            ts = int(tx.get("timeStamp", 0))
            if ts <= cutoff_ts:
                transfers.append(tx)
            else:
                return transfers

        if len(batch) < params["offset"]:
            break
        params["page"] += 1

    return transfers

## Функция объединения собранных данных

In [7]:
def collect_all_to_dataframe(
    addresses: list, api_key: str, end_date: str
) -> pd.DataFrame:
    """
    Собирает normal, internal и ERC-20 транзакции для списка адресов до end_date и возвращает DataFrame.
    Колонки:
      - Все поля транзакции
      - tx_type  : 'normal', 'internal' или 'erc20'
      - address  : адрес кошелька
    """
    rows = []

    for addr in addresses:
        # Normal и internal транзакции
        for tx_type in ("normal", "internal"):
            txs = fetch_transactions(addr, api_key, end_date, tx_type=tx_type)
            for tx in txs:
                tx_record = tx.copy()
                tx_record["tx_type"] = tx_type
                tx_record["address"] = addr
                rows.append(tx_record)

        # ERC-20 переводы
        erc20s = fetch_token_transfers(addr, api_key, end_date)
        for tx in erc20s:
            tx_record = tx.copy()
            tx_record["tx_type"] = "erc20"
            tx_record["address"] = addr
            rows.append(tx_record)

    df = pd.DataFrame(rows)
    return df

# 🦍 Сбор данных с Etherscan

In [8]:
# Параметры
ARBISCAN_API_KEY = ["RR54CDP695RSWU5PGZRC135FDC32DBMXIQ"][0]
END_DATE = "2024-07-04"

list_of_ours_df = []

In [ ]:
for lll in tqdm(range(9, 120)):
    for i in range(100 * lll, 100 * (lll + 1)):
        try:
            ADDRESSES = list_of_addresses_15k[i * 5 : (i + 1) * 5]
            df = collect_all_to_dataframe(ADDRESSES, ARBISCAN_API_KEY, END_DATE)
            list_of_ours_df.append(df)
            time.sleep(0.3)

        except Exception:
            time.sleep(1)
            continue
        print(i, end=" ")

    with open(f"list_of_df_etherscan_offset_100x5_{lll}.pkl", "wb") as f:
        pickle.dump(list_of_ours_df, f)

    list_of_ours_df = []

  0%|          | 0/111 [00:00<?, ?it/s]

900 901 902 903 904 905 906 907 908 909 910 911 912 913 914 915 916 917 918 919 920 921 922 923 924 925 926 927 928 929 930 931 932 933 934 935 936 937 938 939 940 941 942 943 944 945 946 947 948 949 950 951 952 953 954 955 956 957 958 959 960 961 962 963 964 965 966 967 968 969 970 971 972 973 974 975 976 977 978 979 980 981 982 983 984 985 986 987 988 989 990 991 992 993 994 995 996 997 998 

  1%|          | 1/111 [19:38<35:59:48, 1178.08s/it]

999 1000 1001 1002 1003 1004 1005 1006 1007 1008 1009 1010 1011 1012 1013 1014 1015 1016 1017 1018 1019 1020 1021 1022 1023 1024 1025 1026 1027 1028 1029 1030 1031 1032 1033 1034 1035 1036 1037 1038 1039 1040 1041 1042 1043 1044 1045 1046 1047 1048 1049 1050 1051 1052 1053 1054 1055 1056 1057 1058 1059 1060 1061 1062 1063 1064 1065 1066 1067 1068 1069 1070 1071 1072 1073 1074 1075 1076 1077 1078 1079 1080 1081 1082 1083 1084 1085 1086 1087 1088 1089 1090 1091 1092 1093 1094 1095 1096 1097 1098 

  2%|▏         | 2/111 [39:25<35:50:05, 1183.54s/it]

1099 1100 1101 1102 1103 1104 1105 1106 1107 1108 1109 1110 1111 1112 1113 1114 1115 1116 1117 1118 1119 1120 1121 1122 1123 1124 1125 1126 1127 1128 1129 1130 1131 1132 1133 1134 1135 1136 1137 1138 1139 1140 1141 1142 1143 1144 1145 1146 1147 1148 1149 1150 1151 1152 1153 1154 1155 1156 1157 1158 1159 1160 1161 1162 1163 1164 1165 1166 1167 1168 1169 1170 1171 1172 1173 1174 1175 1176 1177 1178 1179 1180 1181 1182 1183 1184 1185 1186 1187 1188 1189 1190 1191 1192 1193 1194 1195 1196 1197 1198 

  3%|▎         | 3/111 [59:19<35:38:42, 1188.17s/it]

1199 1200 1201 1202 1203 1204 1205 1206 1207 1208 1209 1210 1211 1212 1213 1214 1215 1216 1217 1218 1219 1220 1221 1222 1223 1224 1225 1226 1227 1228 1229 1230 1231 1232 1233 1234 1235 1236 1237 1238 1239 1240 1241 1242 1243 1244 1245 1246 1247 1248 1249 1250 1251 1252 1253 1254 1255 1256 1257 1258 1259 1260 1261 1262 1263 1264 1265 1266 1267 1268 1269 1270 1271 1272 1273 1274 1275 1276 1277 1278 1279 1280 1281 1282 1283 1284 1285 1286 1287 1288 1289 1290 1291 1292 1293 1294 1295 1296 1297 1298 

  4%|▎         | 4/111 [1:18:13<34:40:58, 1166.90s/it]

1299 1300 1301 1302 1303 1304 1305 1306 1307 1308 1309 1310 1311 1312 1313 1314 1315 1316 1317 1318 1319 1320 1321 1322 1323 1324 1325 1326 1327 